In [17]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

In [2]:
#stop word removal
from nltk.corpus import stopwords
stop={'a', 'of', 'on', 'the','a','an','the','and','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','only','own','same','so','than','too','very','will','just','should','now' } #add related stop word here
print(stop)

{'and', 'where', 'same', 'will', 'down', 'again', 'most', 'for', 'too', 'the', 'once', 'under', 'then', 'as', 'own', 'below', 'before', 'each', 'any', 'while', 'some', 'an', 'now', 'at', 'about', 'to', 'of', 'into', 'why', 'should', 'how', 'from', 'above', 'over', 'few', 'here', 'than', 'so', 'up', 'between', 'through', 'there', 'against', 'out', 'very', 'a', 'just', 'after', 'until', 'with', 'if', 'both', 'only', 'by', 'during', 'all', 'when', 'more', 'or', 'off', 'on', 'further', 'in', 'other', 'because'}


In [20]:
# df_csv3 = pd.read_csv('2019 olid/olid-training-v1.0.tsv',sep='\t')
df_csv4 = pd.read_csv('task_a_distant_NOT.csv',sep=',')
df_csv5 = pd.read_csv('task_a_distant_OFF.csv',sep=',')
df_test = pd.read_csv('English/test_a_tweets.tsv',sep='\t')
df_csv3.head(10)

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN
7,52415,@USER was literally just talking about this lo...,OFF,TIN,GRP
8,45157,@USER Buy more icecream!!!,NOT,NaN,NaN
9,13384,@USER Canada doesn’t need another CUCK! We alr...,OFF,TIN,IND


In [15]:
#marge all data from files that has been read
twee=[]
ids=[]
for i in range(len(df_test)):
    ids.append(df_test['id'][i])
    twee.append(df_test['tweet'][i])
    
av=[]
st=[]

#marge all data from files that has been read

q_a=[]
label=[]
# for i in range(len(df_csv3)):
#     label.append(df_csv3['subtask_a'][i])
#     q_a.append(df_csv3['tweet'][i])
    
for i in range(len(df_csv4)):
    label.append(df_csv4['label'][i])
    q_a.append(df_csv4['text'][i])



for i in range(len(df_csv5)):
    label.append(df_csv5['label'][i])
    q_a.append(df_csv5['text'][i])
    av.append(df_csv5['average'][i])
    st.append(df_csv5['std'][i])

In [16]:
print(len(q_a))
print(len(label))
print(len(twee))
print(len(ids))
print(max(av))
print(min(av))
print(max(st))
print(min(st))

16619
16619
3887
3887
0.9567039679079568
0.9000045757524653
0.16502006652768367
0.03709859575808765


In [79]:
# remove empty space from the begining of a sentance
TWEETS=[]
for i in twee:
    sen1=i.replace('@USER','')
    sen1=sen1.replace('@user','')
    sen1=sen1.replace("#", ' ')
    sen1=sen1.replace('@USER','')
    sen1=sen1.replace("NEWLINE_TOKEN", " ")
    sen1=sen1.replace("@", "")
    sen1=sen1.replace("TAB_TOKEN", " ")
    sen1=sen1.replace(" w ", ' with ')
    sen1=sen1.replace(" mf ", ' motherfucker ')
    sen1=sen1.replace(" IDGAF ", ' i dont give a fuck ')
    sen1=sen1.replace(" IDRC ", ' I Dont Really Care ')
    sen1=sen1.replace(" tf ", ' the fuck ')
    sen1=sen1.replace(" wtf ", ' what the fuck ')
    sen1=sen1.replace(" yanno ", ' you know ')
    sen1=sen1.replace(" igt ", ' i know right ')
    sen1=sen1.replace(" r ", ' are ')
    sen1=sen1.replace(" tbh ", ' to be honest ')
    sen1=sen1.replace(" ik ", ' i know ')
    sen1=sen1.replace(" af ", ' as fuck ')
    sen1=sen1.replace(" des ", ' this ')
    sen1=sen1.replace(" bout ", ' about ')
    sen1=sen1.replace(" em ", ' them ')
    sen1=sen1.replace(" stg ", ' swear to god ')
    sen1=sen1.replace(" bj ", ' blow job ')
    sen1=sen1.replace(" ig ", ' i guess ')
    sen1=sen1.replace(" fagg ", ' faggot ')
    sen1=sen1.replace(" fag ", ' faggot ')
    sen1=sen1.replace(" tfw ", ' That Feel When ')
    sen1=sen1.replace(" wa ", ' was ')
    sen1=sen1.replace(" n ", ' and ')
    sen1=sen1.replace(" y0 ", ' you ')
    sen1=sen1.replace(" dat ", ' that ')
    sen1=sen1.replace(" YO ", ' you ')
    sen1=" ".join(sen1.split())
    TWEETS.append(sen1)
twee=TWEETS

In [241]:
# convert labeling to 0 and 1
lb=[]
for i in label:
    if i=='OFF':
        lb.append(1)
    if i=='NOT':
        lb.append(0)

print(lb)
labels=lb

[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 

In [227]:
# remove empty space from the begining of a sentance
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=" ".join(sen.split())
    QA.append(sen)
print(QA)
q_a=QA

['She should ask a few native Americans what their take on this is.', 'Go home you’re drunk!!! MAGA Trump2020 👊🇺🇸👊 URL', 'Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL Amazon MAGA KAG CHINA TCOT', 'Someone should\'veTaken" this piece of shit to a volcano. 😂"', 'Obama wanted liberals &amp; illegals to move into red states', 'Liberals are all Kookoo !!!', 'Oh noes! Tough shit.', 'was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism', 'Buy more icecream!!!', 'Canada doesn’t need another CUCK! We already have enough LooneyLeft Liberals f**king up our great country! Qproofs TrudeauMustGo', 'It’s not my fault you support gun control', 'What’s the difference between Kavanaugh and One of these men admitted to groping a 15 year old girl years ago. The other is going to be con

In [131]:
# #this code will stem text
stem_both_q_a=[]
for i in q_a:
    ps = PorterStemmer() #stemming word removing
    words = str(i).split(' ') 
    s=''
    for w in words: 
        stem=ps.stem(w)
        s=s+' '+stem
    s=s.strip()

    stem_both_q_a.append(s)
q_a=stem_both_q_a

# print(q_a)

In [242]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=q_a
comments_attack['label']=labels

In [255]:
dataframe = comments_attack

In [256]:
# Text preprocessing
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace('@USER',''))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" w ", ' with '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" mf ", ' motherfucker '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDGAF ", ' i dont give a fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDRC ", ' I Dont Really Care '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tf ", ' the fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wtf ", ' what the fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" yanno ", ' you know '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" igt ", ' i know right '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" r ", ' are '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tbh ", ' to be honest '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ik ", ' i know '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" af ", ' as fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" des ", ' this '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bout ", ' about '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" em ", ' them '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" stg ", ' swear to god '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bj ", ' blow job '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ig ", ' i guess '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fagg ", ' faggot '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fag ", ' faggot '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tfw ", ' That Feel When '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wa ", ' was '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" n ", ' and '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" y0 ", ' you '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" dat ", ' that '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" YO ", ' you '))



In [230]:
# this code will remove stop word
for c in stop: #stop word removing
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(' '+c+' ', ' '))


In [257]:
print(dataframe['label'])

0        1
1        1
2        0
3        1
4        0
        ..
29854    1
29855    1
29856    1
29857    1
29858    1
Name: label, Length: 29859, dtype: int64


In [246]:
comments_attack['label'][990]==0

True

In [258]:
print(dataframe['comment'])

0        @USER She should ask a few native Americans wh...
1        @USER @USER Go home you’re drunk!!! @USER #MAG...
2        Amazon is investigating Chinese employees who ...
3        @USER Someone should'veTaken" this piece of sh...
4        @USER @USER Obama wanted liberals &amp; illega...
                               ...                        
29854                                        Piece of shit
29855    this afro latino shit is so annoying shut the ...
29856                          fucking disgrace to favelas
29857    Treat a bitch like her credit score poor as fuck!
29858    Fucking getting robbed MARTIN Atkinson ignored...
Name: comment, Length: 29859, dtype: object


In [29]:
# for x in dataframe['comment']:
#     print(x)

In [259]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.2, random_state=42)

In [260]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  23887
test comments length:  5972


In [261]:
# word level tf-idf
# tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# tfidf_vect.fit(dataframe['comment'])
# xtrain_tfidf =  tfidf_vect.transform(train_x)
# xvalid_tfidf =  tfidf_vect.transform(valid_x)

# # ngram level tf-idf 
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
# tfidf_vect_ngram.fit(dataframe['comment'])
# xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
# xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)


In [143]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [262]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [125]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, WordLevel TF-IDF:   accuracy: 0.8921634293369055     f1 score: 0.8894525844114596
NB, N-Gram Vectors:   accuracy: 0.775954454119223     f1 score: 0.753545347531771
NB, CharLevel Vectors:   accuracy: 0.8747488278633624   f1 score: 0.869119125958184


In [263]:
# # Linear Classifier on Word Level TF IDF Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
# print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
# print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR, CharLevel Vectors:   accuracy: 0.8908238446081714   f1 score: 0.8876314213794875


In [147]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [148]:
# SVM Classifier on Count Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count,valid_y)
print("SVM, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("SVM, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("SVM, N-Gram Vectors TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("SVM, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, Count Vectors:   accuracy: 0.8007367716008037   f1 score: 0.7835138049126037


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, WordLevel TF-IDF:   accuracy: 0.6764902880107166   f1 score: 0.5459490138959839


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, N-Gram Vectors TF-IDF:   accuracy: 0.6764902880107166   f1 score: 0.5459490138959839
SVM, CharLevel Vectors:   accuracy: 0.6764902880107166   f1 score: 0.5459490138959839


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [149]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [150]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
23887/23887 [==============================] - 16s 671us/step - loss: 0.4059 - accuracy: 0.8317
Epoch 2/3
23887/23887 [==============================] - 16s 661us/step - loss: 0.2530 - accuracy: 0.8966
Epoch 3/3
23887/23887 [==============================] - 16s 658us/step - loss: 0.2328 - accuracy: 0.9033
CNN, Word Embeddings acuuracy accuracy:0.9022102952003479     f1 score: 0.9016268453975631


In [151]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(3, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [152]:
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
23887/23887 [==============================] - 70s 3ms/step - loss: 0.3936 - accuracy: 0.8414
Epoch 2/3
23887/23887 [==============================] - 77s 3ms/step - loss: 0.2667 - accuracy: 0.8849
Epoch 3/3
23887/23887 [==============================] - 74s 3ms/step - loss: 0.2469 - accuracy: 0.8912
LSTM, Word Embeddings accuracy:0.8995311260223389     f1 score: 0.8962878061266843


In [153]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [154]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# Naive Bayes on Word Level TF IDF Vectors

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# RF on Word Level TF IDF Vectors ngram
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print(accuracy)

# RF on Word Levelon Character Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)

print(accuracy)


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7903549899531145


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7836570663094441


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7314132618888145


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8749162759544541


In [100]:
clf = Pipeline([
    ('vect',tfidf_vect_ngram_chars),
    ('clf', linear_model.LogisticRegression()),
])

clf = clf.fit(train_x, train_y)

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [101]:
Labels=[]
IDE=[]
for i in range(len(twee)):
    IDE.append(ids[i])
    predictions = clf.predict([twee[i]])
    if predictions[0]==1:
        Labels.append('OFF')
    elif predictions[0]==0:
        Labels.append('NOT')
print(len(Labels))
print(len(IDE))

3887
3887


In [102]:
my_dict = { 'q_a' : IDE,'label' : Labels}
df = pd.DataFrame(my_dict)
df.to_csv('English_submission2',index=False)